In [1]:
%load_ext autoreload
%autoreload 1
%matplotlib inline
%aimport utils.remote_execute_util
from utils.mnist_utils import load_data
import os, time, logging
from  azureml.core.runconfig import DataReferenceConfiguration
from azureml.core import Workspace
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from utils.remote_execute_util import prepare_remote_experiment, create_experiment
from utils.aml_util import best_accuracy, get_remote_compute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from sklearn import datasets

## Execute AutoML training remotely

We have already uploaded the dataset into MLSW (in initial setup script)

In [2]:
estimator = prepare_remote_experiment(regularization=0.8)
experiment = create_experiment('mnist')
ws = Workspace.from_config()
ds = ws.get_default_datastore()
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='mnist_data', 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)


WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


found compute target. Using it. cpucluster


In [3]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = get_remote_compute()
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# set the data reference of the run coonfiguration
conda_run_config.data_references = {ds.name: dr}

cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]'], conda_packages=['numpy','py-xgboost<=0.80'])
conda_run_config.environment.python.conda_dependencies = cd

found compute target. Using it. cpucluster


In [6]:

automl_settings = {
    "name": "mnist_automl_{0}".format(time.time()),
    "iteration_timeout_minutes": 10,
    "iterations": 20,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 10,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             run_configuration=conda_run_config,
                             data_script = 'src/get_data.py',
                             **automl_settings,)
run = experiment.submit(automl_config, show_output = True)
run

Running on remote compute: cpucluster


AttributeError: 'AzureAutoMLClient' object has no attribute 'user_script'

## Monitor run
to see real-time information from remote, install: jupyter nbextension install --py --user azureml.widgets

In [5]:
RunDetails(run).show()

NameError: name 'run' is not defined

## Inspect results

In [ ]:
print(run.get_output())

Register model if it was the best so far

In [3]:
accuracy = run.get_metrics()['accuracy']

if (accuracy>best_accuracy('sklearn_mnist')):
    run.register_model(model_name='sklearn_mnist', model_path='outputs/sklearn_mnist_model.pkl',properties={'accuracy': accuracy})

NameError: name 'run' is not defined

In [4]:
best_accuracy('sklearn_mnist')

0.9255


0.9255